In [1]:
import fridge
import fridge.utilities.h5Interface as h5i
import fridge.utilities.reactorInterface as RI
import os

from scipy.interpolate import interp1d as interp
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import numpy as np
import blackboard
import ka
import ka_rp as karp
import ka_br as kabr
import bb_sfr_opt
import time

# SFR Optimization

This example seeks to optimize a SFR design utilizing the blackboard framework.
We start first by taking a look at the SFR design data using the `h5` add `reactor` interfaces built into `FRDIGe` [cite].
For this we'll examine the objectives that will drive the optimization process: cycle length, plutonium mass, reactivity swing, and core average burnup.
Upon examination, we note that we have removed a significant portion of the design space by ensuring that the reactor design has a cycle length greater than 0 days.

In [4]:
h5_interface = h5i.h5Interface()
h5_interface.read_h5('/Users/ryanstewart/projects/sfr_database/SFR_DB.h5')

In [5]:
cycle_length_list = []
rx_swing_list = []
height = []
smear = []
pu_content = []
bu = []
pu_mass = []
for rx, core in h5_interface.h5file.items():
    try:
        rx_ = RI.reactorInterface(core)
        cycle_length = rx_.extrapolate_value('keff', 'time', 1.0)
        rx_swing = rx_.get_reactivity_swing(1.0, cycle_length)
        rx_swing_per_day = rx_swing/cycle_length
        avg_bu = rx_.get_assembly_avg(cycle_length, 'burnup')
        avg_bu_per_day = avg_bu/cycle_length
        pu = rx_.get_assembly_pu_mass() * 78 / 4
        if cycle_length > 0:
            height.append(rx_.rx['independent variables']['height'][0])
            smear.append(rx_.rx['independent variables']['smear'][0])
            pu_content.append(rx_.rx['independent variables']['pu_content'][0])
            cycle_length_list.append(cycle_length)
            rx_swing_list.append(rx_swing)
            bu.append(avg_bu)
            pu_mass.append(pu)
    except (KeyError, TypeError):
        print('didnt get {}'.format(core))

In [4]:
fig = px.scatter_3d(x=height, y=smear, z=cycle_length_list, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Cycle Length (days)','color':'Pu Fraction'})
fig.show()
fig2 = px.scatter_3d(x=height, y=smear, z=rx_swing_list, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Rx Swing (pcm)','color':'Pu Fraction'})
fig2.show()
fig3 = px.scatter_3d(x=height, y=smear, z=bu, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Burnup (GWd/MTU)','color':'Pu Fraction'})
fig3.show()
fig4 = px.scatter_3d(x=height, y=smear, z=pu_mass, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Pu Mass per Cycle (kg)','color':'Pu Fraction'})
fig4.show()

NameError: name 'height' is not defined

# Starting the Blackboard System

In [2]:
ns = run_nameserver()
bb = run_agent(name='bb', base=bb_sfr_opt.BbSfrOpt)
bb.set_attr(desired_results={'cycle length': (365, 1500), 
                             'reactivity swing': (0, 7500), 
                             'burnup': (0,175), 
                             'pu mass': (0, 1750)})
bb.generate_sm()

bb.connect_agent(karp.KaRpExplore, 'ka_rp_explore_1')
bb.connect_agent(karp.KaRpExplore, 'ka_rp_explore_2')
bb.connect_agent(karp.KaRpExploit, 'ka_rp_exploit')
bb.connect_agent(kabr.KaBr_lvl2, 'ka_br_lvl2')
bb.connect_agent(kabr.KaBr_lvl3, 'ka_br_lvl3')

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:12307 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:12307
INFO [2020-05-04 22:09:31.270104] (bb): Connected agent ka_rp_explore_1 of agent type <class 'ka_rp.KaRpExplore'>
INFO [2020-05-04 22:09:31.323301] (bb): Connected agent ka_rp_explore_2 of agent type <class 'ka_rp.KaRpExplore'>
INFO [2020-05-04 22:09:31.376822] (bb): Connected agent ka_rp_exploit of agent type <class 'ka_rp.KaRpExploit'>
INFO [2020-05-04 22:09:31.422976] (bb): Connected agent ka_br_lvl2 of agent type <class 'ka_br.KaBr_lvl2'>
INFO [2020-05-04 22:09:31.471617] (bb): Connected agent ka_br_lvl3 of agent type <class 'ka_br.KaBr_lvl3'>
INFO [2020-05-04 22:09:34.488447] (bb): Selecting agent ka_rp_explore_1 (TV: 1.0) to execute (TE: 1)
INFO [2020-05-04 22:09:37.499649] (bb): Selecting agent ka_rp_explore_2 (TV: 1.0) to execute (TE: 2)
INFO [2020-05-04 22:09:40.513421] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 3)
INFO [2020-05-04 22:0

INFO [2020-05-04 22:11:59.076819] (bb): Selecting agent ka_rp_exploit (TV: 11.0) to execute (TE: 49)
INFO [2020-05-04 22:12:02.096929] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 50)
INFO [2020-05-04 22:12:02.100925] (bb): Writing blackboard to archive
INFO [2020-05-04 22:12:08.112177] (bb): Finished writing to archive
INFO [2020-05-04 22:12:09.650141] (ka_br_lvl3):  Number of entries in 
 Level 3: 28, 27 
 Level 2: 2, 24 
 Level 1: 1, 8
INFO [2020-05-04 22:12:11.124109] (bb): Selecting agent ka_br_lvl2 (TV: 10.0) to execute (TE: 51)
INFO [2020-05-04 22:12:14.136271] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 52)
INFO [2020-05-04 22:12:14.670437] (ka_br_lvl3):  Number of entries in 
 Level 3: 27, 28 
 Level 2: 1, 25 
 Level 1: 1, 8
INFO [2020-05-04 22:12:17.149628] (bb): Selecting agent ka_rp_exploit (TV: 11.0) to execute (TE: 53)
INFO [2020-05-04 22:12:20.171651] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 54)
INFO [2020-05-04 22:12:23.

INFO [2020-05-04 22:14:44.786098] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 98)
INFO [2020-05-04 22:14:45.339417] (ka_br_lvl3):  Number of entries in 
 Level 3: 50, 53 
 Level 2: 1, 48 
 Level 1: 1, 16
INFO [2020-05-04 22:14:47.799195] (bb): Selecting agent ka_rp_exploit (TV: 11.0) to execute (TE: 99)
INFO [2020-05-04 22:14:50.827421] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 100)
INFO [2020-05-04 22:14:50.830395] (bb): Writing blackboard to archive
INFO [2020-05-04 22:14:56.843216] (bb): Finished writing to archive
INFO [2020-05-04 22:14:59.424359] (ka_br_lvl3):  Number of entries in 
 Level 3: 54, 54 
 Level 2: 1, 49 
 Level 1: 0, 17
INFO [2020-05-04 22:14:59.853139] (bb): Selecting agent ka_rp_explore_1 (TV: 1.0) to execute (TE: 101)
INFO [2020-05-04 22:15:02.866487] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 102)
INFO [2020-05-04 22:15:03.423025] (ka_br_lvl3):  Number of entries in 
 Level 3: 54, 55 
 Level 2: 1, 50 
 Level 1: 0,

INFO [2020-05-04 22:17:24.546289] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 147)
INFO [2020-05-04 22:17:25.122898] (ka_br_lvl3):  Number of entries in 
 Level 3: 73, 82 
 Level 2: 1, 73 
 Level 1: 1, 24
INFO [2020-05-04 22:17:27.555382] (bb): Selecting agent ka_rp_exploit (TV: 11.0) to execute (TE: 148)
INFO [2020-05-04 22:17:30.591637] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 149)
INFO [2020-05-04 22:17:33.598040] (bb): Selecting agent ka_rp_explore_1 (TV: 1.0) to execute (TE: 150)
INFO [2020-05-04 22:17:33.713545] (ka_br_lvl3):  Number of entries in 
 Level 3: 77, 83 
 Level 2: 1, 74 
 Level 1: 0, 25
INFO [2020-05-04 22:17:36.609518] (bb): Selecting agent ka_br_lvl2 (TV: 10.0) to execute (TE: 151)
INFO [2020-05-04 22:17:39.618928] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 152)
INFO [2020-05-04 22:17:40.189506] (ka_br_lvl3):  Number of entries in 
 Level 3: 77, 84 
 Level 2: 1, 75 
 Level 1: 1, 25
INFO [2020-05-04 22:17:42.628301]

INFO [2020-05-04 22:20:07.380849] (bb): Selecting agent ka_rp_explore_1 (TV: 1.0) to execute (TE: 197)
INFO [2020-05-04 22:20:07.499385] (ka_br_lvl3):  Number of entries in 
 Level 3: 104, 107 
 Level 2: 1, 97 
 Level 1: 0, 33
INFO [2020-05-04 22:20:10.394546] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 198)
INFO [2020-05-04 22:20:12.498279] (ka_br_lvl3):  Number of entries in 
 Level 3: 103, 109 
 Level 2: 1, 98 
 Level 1: 0, 33
INFO [2020-05-04 22:20:13.403454] (bb): Selecting agent ka_br_lvl2 (TV: 10.0) to execute (TE: 199)
INFO [2020-05-04 22:20:16.415177] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 200)
INFO [2020-05-04 22:20:17.013559] (ka_br_lvl3):  Number of entries in 
 Level 3: 102, 110 
 Level 2: 1, 99 
 Level 1: 1, 33
INFO [2020-05-04 22:20:19.428301] (bb): Selecting agent ka_rp_exploit (TV: 11.0) to execute (TE: 201)
INFO [2020-05-04 22:20:22.474755] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 202)
INFO [2020-05-04 22:20:25.4

INFO [2020-05-04 22:22:47.242343] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 244)
INFO [2020-05-04 22:22:47.854817] (ka_br_lvl3):  Number of entries in 
 Level 3: 121, 136 
 Level 2: 1, 121 
 Level 1: 1, 40
INFO [2020-05-04 22:22:50.254076] (bb): Selecting agent ka_rp_exploit (TV: 11.0) to execute (TE: 245)
INFO [2020-05-04 22:22:53.308437] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 246)
INFO [2020-05-04 22:22:56.318226] (bb): Selecting agent ka_rp_explore_1 (TV: 1.0) to execute (TE: 247)
INFO [2020-05-04 22:22:56.503228] (ka_br_lvl3):  Number of entries in 
 Level 3: 125, 137 
 Level 2: 1, 122 
 Level 1: 0, 41
INFO [2020-05-04 22:22:59.331763] (bb): Selecting agent ka_br_lvl2 (TV: 10.0) to execute (TE: 248)
INFO [2020-05-04 22:23:02.342481] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 249)
INFO [2020-05-04 22:23:02.952214] (ka_br_lvl3):  Number of entries in 
 Level 3: 125, 138 
 Level 2: 1, 123 
 Level 1: 1, 41
INFO [2020-05-04 22:23:0

INFO [2020-05-04 22:25:21.212408] (bb): Selecting agent ka_br_lvl2 (TV: 10.0) to execute (TE: 295)
INFO [2020-05-04 22:25:24.223464] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 296)
INFO [2020-05-04 22:25:24.851939] (ka_br_lvl3):  Number of entries in 
 Level 3: 149, 163 
 Level 2: 1, 146 
 Level 1: 1, 49
INFO [2020-05-04 22:25:27.237285] (bb): Selecting agent ka_rp_exploit (TV: 11.0) to execute (TE: 297)
INFO [2020-05-04 22:25:30.304925] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 298)
INFO [2020-05-04 22:25:33.316436] (bb): Selecting agent ka_rp_explore_1 (TV: 1.0) to execute (TE: 299)
INFO [2020-05-04 22:25:33.482033] (ka_br_lvl3):  Number of entries in 
 Level 3: 151, 165 
 Level 2: 1, 147 
 Level 1: 0, 50
INFO [2020-05-04 22:25:36.328965] (bb): Selecting agent ka_br_lvl3 (TV: 10.0) to execute (TE: 300)
INFO [2020-05-04 22:25:36.964843] (ka_br_lvl3):  Number of entries in 
 Level 3: 151, 166 
 Level 2: 2, 147 
 Level 1: 0, 50
INFO [2020-05-04 22:25:3

In [3]:
while not bb.get_attr('_complete'):
    bb.publish_trigger()
    time.sleep(3)
    bb.controller()
    bb.send_executor()
    bb.wait_for_ka()
    bb.determine_complete()
    print('Finished trigger event: {}'.format(bb.get_attr('_trigger_event')))    

Finished trigger event: 1
Finished trigger event: 2
Finished trigger event: 3
Finished trigger event: 4
Finished trigger event: 5
Finished trigger event: 6
Finished trigger event: 7
Finished trigger event: 8
Finished trigger event: 9
Finished trigger event: 10
Finished trigger event: 11
Finished trigger event: 12
Finished trigger event: 13
Finished trigger event: 14
Finished trigger event: 15
Finished trigger event: 16
Finished trigger event: 17
Finished trigger event: 18
Finished trigger event: 19
Finished trigger event: 20
Finished trigger event: 21
Finished trigger event: 22
Finished trigger event: 23
Finished trigger event: 24
Finished trigger event: 25
Finished trigger event: 26
Finished trigger event: 27
Finished trigger event: 28
Finished trigger event: 29
Finished trigger event: 30
Finished trigger event: 31
Finished trigger event: 32
Finished trigger event: 33
Finished trigger event: 34
Finished trigger event: 35
Finished trigger event: 36
Finished trigger event: 37
Finished t

Finished trigger event: 298
Finished trigger event: 299
Finished trigger event: 300
Finished trigger event: 301
Finished trigger event: 302
